In [ ]:
import pandas as pd
import os
from datasets import Dataset
from transformers import AutoTokenizer

current_dir = os.getcwd()
print(current_dir)
if current_dir.endswith("week4"):
    os.chdir("../..")
    print(os.getcwd())
else:
    print("current dir", current_dir)


In [ ]:

ds_train = pd.read_parquet("dataset/train_df.parquet")
ds_val = pd.read_parquet("dataset/val_df.parquet")


In [ ]:

drop_cols = [col for col in ds_train.columns if col not in ["question", "context", "answer"]]

train_jsonl = ds_train.drop(columns=drop_cols)
train_jsonl.to_json("code/week4/data/finetune_data.jsonl", orient="records", lines=True)
#val_jsonl = ds_val.drop(columns=drop_cols)
#val_jsonl.to_json("code/week4/data/val_data.jsonl", orient="records", lines=True)

In [ ]:
from typing import Any, List, Optional


remove_sentence = """Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024"""

def construct_prompt(
    tokenizer : AutoTokenizer,
    question : str,
    context : str,
    answer : Optional[str] = None,
    tokenize : bool = False
) -> str:
    
    messages = [
        {"role": "system", "content": "Given the users context and question, answer the question."},
        {"role": "user", "content": f"Question: {question}\nContext: {context}"}
    ]
    if answer:
        messages.append({"role": "assistant", "content": f"{answer}"})
    prompt = tokenizer.apply_chat_template(messages, tokenize=tokenize, add_generation_prompt=False)
    return prompt


import re
    
def prompt_to_segments(
    prompt: str,
) -> List[dict[str, Any]]:
    """
    Convert a prompt to a list of segments.
    """
    
    middle_text = 'assistant<\|end_header_id\|>'
    end_text = '<\|eot_id\|>'
    pattern = r'(.*?)assistant<\|end_header_id\|>(.*?)<\|eot_id\|>'
    match = re.search(pattern, prompt, re.DOTALL)
    chunks = [
        {"text": match.group(1), "label": "false"},
        {"text": middle_text, "label": "false"},
        {"text": match.group(2), "label": "true"},
        {"text": end_text, "label": "false"}
    ]

    return chunks

import json
def construct_input_output_from_df(
    df: pd.DataFrame,
    tokenizer : AutoTokenizer,
    write_path : str
):
    df['prompt_str'] = df.apply(
        lambda x: 
            prompt_to_segments(
                construct_prompt(
                    tokenizer, x['question'], x['context'], x['answer'], tokenize=False
                )
            ), axis=1)
    
    _dict = {'segments': df['prompt_str'].tolist()}
    with open(write_path, 'w') as f:
        json.dump(_dict, f)
    
construct_input_output_from_df(ds_train, tokenizer, "code/week4/data/finetune_data.jsonl")

